# 課題の概要

この宿題では、講義で学んだRAG(Retrieval-Augmented Generation)技術を用いて、LLMの生成
内容を改善する実践的な取り組みを行います。演習で利用したコードをベースに、独自の質問と参照文書を用
いて実験を行い、RAGの効果を定量的・定性的に評価します。
この宿題を通じて、「テストデータの作成」と「改善のプロセス」について理解を深め、実際のアプリケーション開発
に役立てることを目指します。



## 内容

宿題の内容
1. 独自の質問と参照資料の作成
- 自分で5つ以上の質問文を考案してください
- 各質問に対する回答を含む参照文書を用意してください
- 少なくとも1つは、LLMが単体では正確に答えられないような知識を含む質問にしてください
2. 実験の実施
- 演習で使用したコードをベースに、以下の2つの方法で回答を生成してください
  - ベースのLLM(RAGなし)での回答生成
  -  RAGを組み合わせた回答生成
- 回答の評価では、単純なYes/No判定でも良いです
  - より詳細な評価指標も検討していただけるとなお良いです

3. 結果分析と考察

- 生成した結果をまとめ、RAGありとRAGなしの差異を分析してください
- RAGによって回答が改善したケースと悪化したケースの両方について考察してください- 結果に基づいて、RAGの有効性と限界についての考察を記述してください

## 扱う条件

**<使用するモデル>**

2024年4月リリースのMeta-Llama-3-8B-Instructを使用する


**<参考文献>**

・ディズニー公式

・Wikipedia

・[映画『インサイド・ヘッド』あらすじ&ネタバレ！キャラクター、声優キャスト、トリビアも♪
](https://castel.jp/p/4118)


・[アナと雪の女王のあらすじをネタバレ解説!最後の結末と感動の感想まで!](https://up-tsukuba.com/frozen-summary/)

・[【作品、歴史から見る】ディズニーとピクサーの違いとは](https://liclog.net/disney-pixar-difference/)


・[映画「インサイド・ヘッド２」ネタバレあらすじ結末と感想](https://eiga-watch.com/inside-out-2/)


## 質問概要


・ディズニーに関する質問


1. アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？

  解答：オラフ

2. 「アナと雪の女王」でエピソードが展開される国の名前は何ですか？

  解答：アレンデール



3. ディズニー映画「インサイドヘッド」に関して教えて

  期待する解答：

  主人公少女ライリーの頭の中にいるヨロコビ、カナシミ、イカリ、ムカムカ、ビビリ5つの感情が繰り広げる冒険と1人の少女の成長を描いたストーリー

4. ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて

  解答：
  
  ヨロコビ、カナシミ、イカリ、ムカムカ、ビビリ

  シンパイ、イイナ、ハズカシ、ダリィ、ナツカシ

5. ディズニーとピクサーの違いを教えて

  解答の評価ポイント：

  設立年

  共通する点

  例)

  親会社が、ウォルトディズニーカンパニーであることなど

  異なる点

  例)

  名前・コンセプトなど

  この二つが明確にわけられているかどうかを判断基準として定性的に評価する}

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

In [ ]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=400,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

## 1～5に関して質問を行う


### 1 アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 2.「アナと雪の女王」でエピソードが展開される国の名前は何ですか？

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "「アナと雪の女王」でエピソードが展開される国の名前は何ですか？"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 3.ディズニー映画「インサイドヘッド」に関して教えて

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニー映画「インサイドヘッド」に関して教えて"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 4.ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニー映画ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

### 5 ディズニーとピクサーの違いを教えて

In [ ]:
def generate_output2(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "ディズニーとピクサーの違いを教えて"
response = generate_output2(question, system_prompt)

In [ ]:
response

# 結果


**1. アナと雪の女王」に登場する、魔法で作られた雪だるまの名前はなに？**

回答：

「アナと雪の女王」シリーズの魔法で作られた雪だるまの名前は「サマンサ」。

**2.「アナと雪の女王」でエピソードが展開される国の名前は何ですか？**

回答：

アナと雪の女王」シリーズでは、エピソードが展開される国の名前は「アレンデール」です。


**3.ディズニー映画「インサイドヘッド」に関して教えて**

回答：

「インサイドヘッド」は、2015年に公開されたアメリカン・アニメーション映画「Inside Out」です。監督はピーター・ドクター、製作はピクサー・アニメーション・スタジオ。

この映画は、11歳の少女メイの脳内で起きる想像上の世界を描きます。メイの脳内には5つのエモーション（感情）が存在し、それぞれが彼女の生活に影響を与えています。5つのエモーションは、ジョー（喜び）、バリー（怒り）、ピーター（悲しみ）、サリー（哀しみ）、リグレット（後悔）です。

この映画は、子供の心の世界を描き、子供たちが経験する様々な感情を描きました。同時に、映画は、子供たちが成長する過程で必要な感情のバランスを描きました。

「インサイドヘッド」は、2015年のアカデミー賞で長編アニメーション映画賞を受賞し、世界中で大きな人気を博しました。


**4.ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて**

回答：

「Inside Out」は2015年に公開されたピクサー製作のアニメーション映画です。映画の主人公は11歳の少女、リリーの脳内に存在する5つの感情のキャラクターです。

以下は、映画「Inside Out」に登場する感情のキャラクターの名前と性格です。

1. **Joy**（ハッピー）：リリーの脳内で最も強い感情。彼女は常にリリーを笑顔で見つめ、彼女の幸福を目指します。
2. **Sadness**（哀しみ）：リリーの脳内で2番目の強い感情。彼女はリリーの悲しみを理解し、彼女を慰めるために努力します。
3. **Anger**（怒り）：リリーの脳内で3番目の強い感情。彼はリリーの怒りを燃やし、彼女を守るために戦うします。
4. **Fear**（恐怖）：リリーの脳内で4番目の強い感情。彼女はリリーの恐怖を理解し、彼女を守るために警戒します。
5. **Disgust**（嫌悪）：リリーの脳内で5番目の強い感情。彼女はリリーの嫌悪を理解し、彼女を守るために警戒します。

これらのキャラクターは、リリーの脳内で活動し、彼女の感情を支配しています。映画「Inside Out」は、子供の感情の世界を描き、子供たちに自分の感情を理解することを教えることを目指しています。


**5.ディズニーとピクサーの違いを教えて**


回答：

ディズニーとピクサーは、両方ともアニメーション製作会社ですが、歴史的には独立した存在で、現在でも異なる組織体制を持っています。\n\nディズニーは、ウォルト・ディズニーが創業したアニメーション製作会社で、1930年代に設立されました。ディズニーは、ウォルト・ディズニー自身が創作したキャラクター「ミッキー・マウス」や「ドナルド・ダック」などの人気キャラクターを生み出し、映画「白雪姫」や「シンデレラ」などのクラシック・アニメーション映画を製作しました。\n\n一方、ピクサーは、スティーブ・ジョブズとジョン・ラタザンが1986年に設立したアニメーション製作会社です。ピクサーは、初期にはディズニーとの提携関係にあり、「トイ・ストーリー」や「モンスターズ・インク」などの映画を製作しました。2006年にディズニーがピクサーを買収し、現在はディズニーの一部門として運営されています。\n\nピクサーの特徴として、ストーリー性の強いアニメーション映画を製作し、「トイ・ストーリー」や「インクレディブルズ」などの大ヒット映画を生み出しています。また、ピクサーは、CGI（コンピューター・ジェネレーテッド・イメージ）技術を積極的に使用し、3Dアニメーション映画を製作しています。\n\n以上のように、ディズニーとピクサーは、歴史的には独立した存在で、現在でも異なる組織体制を持っています。ただし、ピクサーがディズニーに買収されたことで、両方の会社は、より強力なアニメーション製作能力を有することになりました。



## 考察

各、質問に対する回答として2や5に関しては正確な回答が得られているが他の解答に関してはところどころに間違いがあり回答としてはあまりいいものとは言えない

また、5に関しては質問の種類がYes/Noではなくかつ抽象的な質問だったのでmax_new_tokensを変更したがそこまで差異はなかった

補足

2に関しては正解してしまったのでRAGでの質問はしないことにした

# RAG実装

### アナ雪

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: Wikipediaにおける「アナと雪の女王」のストーリーを参照
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

In [ ]:
with open("/content/frozen.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[42])

In [ ]:
# Retrievalの実行
question = "アナと雪の女王」に登場する、雪だるまの名前はなに？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] アナと雪の女王」に登場する、雪だるまの名前はなに？"
response = generate_output(question, system_prompt)

In [ ]:
response

### インサイドヘッド


In [ ]:
with open("/content/inside out.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[42])

In [ ]:
# Retrievalの実行
question = "ディズニー映画「インサイドヘッド」に関して教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] ディズニー映画「インサイドヘッド」に関して教えて"
response = generate_output2(question, system_prompt)

In [ ]:
print(response)

まあ全体名流れはだいぶ抑えているので及第点

ただインサイドヘッド2の内容も入っており、冒頭の部分が違和感のあるものになっているのがよくない

In [ ]:
# Retrievalの実行
question = "ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて"
response = generate_output2(question, system_prompt)

In [ ]:
print(response)

### ディズニーとピクサーの違い


In [ ]:
with open("/content/diff_disnet and pixer.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[42])

In [ ]:
# Retrievalの実行
question = "ディズニーとピクサーの違いについて教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] ディズニーとピクサーの違いについて教えて"
response = generate_output(question, system_prompt)

In [ ]:
print(response)

# 結果

**アナ雪**

回答:

参考資料に基づいて、雪だるまの名前はスヴェンです。スヴェンはクリストフの忠実なトナカイです。


**インサイヘッド**

質問

ディズニー映画「インサイドヘッド」に関して教えて

回答：

日本語で回答します。

「インサイド・ヘッド」はディズニー映画の題材であり、前作の出来事から2年が経過した後、ヨロコビとカナシミが「特別な思い出」が散らばり、「長期記憶の保管場所」へ吸い込まれるというストーリーが展開されます。

その後、ヨロコビとカナシミは「記憶のゴミ捨て場」で迷路のように複雑な「長期記憶の保管場所」へ飛ばされ、「性格の島」から司令部へ帰ろうとするが、途中で「友情の島」に到達し、新しい友達ができたと知ったライリーがカッとなった瞬間、「友情の島」まで崩れ落ちてしまう。

ヨロコビは絶望に暮れながら手元の特別な思い出を眺めていると、ある思い出に悲しみと喜びが混じっていることに気付くという展開が描かれます。

質問

ディズニー映画インサイドヘッドで現在までに登場している感情のキャラクターを教えて

回答：

インサイド・ヘッドの前作で登場した5つの感情のキャラクターは、以下の通りです。

1. ヨロコビ (Joy) - ポジティブで前向きの感情
2. カナシミ (Sadness) - ネガティブで悲しい感情
3. イカリ (Anger) - 短気で怒りっぽい感情
4. ムカムカ (Disgust) - 嫌なことを遠ざけるのが役割の感情
5. ビビリ (Fear) - 危険や恐怖から身を守ることが役割の感情

また、新たに登場した感情のキャラクターとして、以下の5つが挙げられます。

1. シンパイ (Anxiety) - 不安を司る感情
2. イイナー (Envy) - 嫉妬を司る感情
3. ハズカシ (Shame) - 羞恥心を司る感情
4. ダリィ (Boredom) - 倦怠感を司る感情

以上、インサイド・ヘッドの感情のキャラクターを紹介しました。


**ディズニーとピクサーの違い**

回答：

ディズニーとピクサーの違いは、製作スタジオ(会社)の違いです。正式には、『ウォルト・ディズニー・アニメーション・スタジオ』と『ピクサー・アニメーション・スタジオ』の違いです。両者は同じ『ウォルト・ディズニー・カンパニー』を親としている子会社同士ですが、映画を製作するスタジオ(会社)としては別の存在です。





# 実装の感想


全体として、RAG実装前よりも正確な情報かつ入れたテキストファイルにもよるが正確な結果を返していることがわかりRAG手法が効果的であることが分かった

ただ、アナ雪のオラフのようにテキストファイルの中に似た属性をもつものが存在するとどうしてもより質問に対する類似度が高い方を優先してしまう傾向があることが分かったのでRAGの実装方法及び、テキストファイル内の情報の均一性などが非常に重要であると感じた

また、ただ参照してきたサイトの文書を丸ごと持ってきただけなのにもかかわらず「インサイドヘッド」のようにキャラクターの出現を明確に分けることもできるようになっておりその点に関してはLLMのすごさを感じたのでこれから実際に自分が内部でなにを行っているかなどをしっかりと追っていけるようになりたいと感じた


# これから

今回の実装で学んだことは主に下記の二つ

1. RAGの実装方法の多様さと懸念点

2. LLMへの質問を明確にする根拠


この二つを今回自分で実装することでさらに、理解が深まりました

特に、最後の方ではなかなかうまくいかない点もあったので授業資料を負いつつほかの講座もとることでなんとか解決手法をこれから探していければと感じました

RAGに関していえば、具体的にはどのような情報を読み込ませることが重要なのだろうかもしくは莫大な量のテキストファイルさえあればいいのかなど基本的なLLMの知識がないのでそこをまず学習しようと感じました

また、自分自身がLLMを実装する機会もしくはさらに先のAIモデルを実装することがあるかもしれないのでその起点になった気がしました

